In [ ]:
#If interrupted during xml editing, data loss is a possibility since we are writing onto file 
#after storing the extracted values. If interrupted after writing onto xml, the stored values are lost.

In [1]:
import cv2
import distutils
from glob import glob
import io
import numpy as np
import os
import re
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.cElementTree as ET
from xml.etree import ElementTree
import xml.dom.minidom as DM
from xml.parsers.expat import ExpatError

In [7]:
from ipynb.fs.full.Data_Prepare import move_given_files, Data_Prepare, delete_small_folders,batch_move_files

In [2]:
def Data_Preprocess(source_dir,destination_dir,disparity_dir,image_extension_list,annotation_extension_list,label_folder_list,forbidden,text="UNIQUENAME"):
    Data_Prepare(source_dir,disparity_dir,image_extension_list,annotation_extension_list,label_folder_list,text)#loop 4
    try:
        shutil.rmtree(destination_dir)
    except FileNotFoundError:
        pass
    shutil.copytree(source_dir,destination_dir)    
    delete_small_folders(destination_dir,4) #loop 5
    classes=next(os.walk(destination_dir))[1]
    for i in forbidden:
        if(i in classes):
            classes.remove(i)
    os.makedirs(destination_dir +'train',exist_ok=True)
    os.makedirs(destination_dir +'test',exist_ok=True)
    train_dir = destination_dir+"train/"
    test_dir = destination_dir+"test/"
    #This only works for xml files. So after class cleanup, call annotation conversion function.
    for i in classes: #loop 6
        class_dir=destination_dir+i+"/"
        #WE NEED TO ONLY MOVE XML FILES TO TRAIN TEST SINCE THE MODEL (detecto) 
        #IS USING PASCAL VOC XML FORMAT. TRAINING, TESTING FOLDERS WILL BE HAVING IMAGES AND XML FILES ONLY. 
        #WE CAN CHANGE IT INTO CSV EASILY USING DETECTO's OWN XML_TO_CSV CONVERTOR FOR TRAIN,TEST AT THE END IF WE EVER NEED TO.
        #THEREFORE TXT, JSON NEEDS TO BE CONVERTED TO XML BEFORE COPY_PROPER STAGE.
        image_files = glob(class_dir+"*.xml")
        #image_files.extend(glob(class_dir+"*.txt"))
        #image_files.extend(glob(class_dir+"*.json"))
        image_files=[name.replace(class_dir[:-1]+"\\","") for name in image_files]
        #loop across annotation_extension_list
        image_names = [name.replace(".xml","") for name in image_files]
        #image_names.extend([name.replace(".txt","") for name in image_files])
        #image_names.extend([name.replace(".json","") for name in image_files])
        #Since train_test_split done for each class, all classes will be equally represented in the 
        #training, testing set.
        #BUT the final configuration of training testing won't be in the ratio mentioned 
        #below unless each class have an equal number of valid images/annotations
        train_names, test_names = train_test_split(image_names, test_size=0.2)
        batch_move_files(test_names, class_dir, test_dir)
        batch_move_files(train_names, class_dir, train_dir)
    delete_small_folders(destination_dir)

In [4]:
#Root directory containing the original dataset
ROOTDIR= 'D:/Downloads/Analysed/Dataset 1/'
CLASSES=next(os.walk(ROOTDIR))[1]
#Files that are not able to make it to the training stage are moved here.
#Reasons maybe because bounding boxes are false, image file doesn't have an annotation file along with it.
#Annotation file doesn't have an image file along with it, files with improper extensions and so on. 
disparity_dir='D:/Downloads/Analysed/Disparity/'
os.makedirs(disparity_dir,exist_ok=True)
#Directory where we will copy proper files and where train and test will be created. 
destination_dir='D:/Downloads/Analysed/DESTINATION/'
#os.makedirs(destination_dir,exist_ok=True)
#Directory to simply experiment around
#root_dir='D:/Downloads/Analysed/E2 - Copy/'
#classes=next(os.walk(root_dir))[1]
#Extension list is the list of file types available in the dataset.
extension_list=['.jpg', '.xml', '.jpeg', '.png', '', '.JPG', '.json', '.webp', '.html', '.txt', '.cms', '.gif', '.jfif']
#Image extension list contains the types of images available in the dataset
#.gif removed since model doesn't read it
image_extension_list=['.jpg', '.png', '.jpeg', '.JPG', '.webp', '.cms', '.jfif'] 
#Annotation extension list contains the types of annotations available in the dataset
annotation_extension_list=['.xml', '.txt', '.json']
#label_folder_list contains the folder names available in some classes which 
#contain the annotations of that class
label_folder_list=['labes','label','annotation','annotations','labels']
forbidden=['train','test']

In [9]:
Data_Preprocess(ROOTDIR,destination_dir,disparity_dir,image_extension_list,annotation_extension_list,label_folder_list,forbidden,"UNIQUENAME")

In  D:/Downloads/Analysed/DESTINATION/Aaloo Chips
In  D:/Downloads/Analysed/DESTINATION/Aamla Murabba
In  D:/Downloads/Analysed/DESTINATION/Aamras
In  D:/Downloads/Analysed/DESTINATION/Akki Roti
In  D:/Downloads/Analysed/DESTINATION/Almond
In  D:/Downloads/Analysed/DESTINATION/Aloo Chokha
In  D:/Downloads/Analysed/DESTINATION/Aloo Chop
In  D:/Downloads/Analysed/DESTINATION/Aloo Gobi Dry
In  D:/Downloads/Analysed/DESTINATION/Aloo Paratha
In  D:/Downloads/Analysed/DESTINATION/Aloo Pitika
In  D:/Downloads/Analysed/DESTINATION/Aloo Potol Posto
In  D:/Downloads/Analysed/DESTINATION/Aloo Shimla
In  D:/Downloads/Analysed/DESTINATION/Aloo Tikki
In  D:/Downloads/Analysed/DESTINATION/Alu Vadi
In  D:/Downloads/Analysed/DESTINATION/Amarnath
In  D:/Downloads/Analysed/DESTINATION/Amritsari Fish
In  D:/Downloads/Analysed/DESTINATION/Apple
In  D:/Downloads/Analysed/DESTINATION/Apple Cider
In  D:/Downloads/Analysed/DESTINATION/Apple Juice
In  D:/Downloads/Analysed/DESTINATION/Apple Pie
In  D:/Downloads

In  D:/Downloads/Analysed/DESTINATION/Paneer Pakoda
In  D:/Downloads/Analysed/DESTINATION/Paneer Seekh Kebab
In  D:/Downloads/Analysed/DESTINATION/Pani Puri
In  D:/Downloads/Analysed/DESTINATION/Panipuri
In  D:/Downloads/Analysed/DESTINATION/Panjiri
In  D:/Downloads/Analysed/DESTINATION/Panneer Rool
In  D:/Downloads/Analysed/DESTINATION/Papaya
In  D:/Downloads/Analysed/DESTINATION/Paratha
In  D:/Downloads/Analysed/DESTINATION/Parwal
In  D:/Downloads/Analysed/DESTINATION/Passion Fruit
In  D:/Downloads/Analysed/DESTINATION/Pastry
In  D:/Downloads/Analysed/DESTINATION/Patishapta
In  D:/Downloads/Analysed/DESTINATION/Pav
In  D:/Downloads/Analysed/DESTINATION/Pav Bhaji
In  D:/Downloads/Analysed/DESTINATION/Pear
In  D:/Downloads/Analysed/DESTINATION/Peda
In  D:/Downloads/Analysed/DESTINATION/Peking Duck
In  D:/Downloads/Analysed/DESTINATION/Pepperoni Pizza
In  D:/Downloads/Analysed/DESTINATION/Petha
In  D:/Downloads/Analysed/DESTINATION/Pho
In  D:/Downloads/Analysed/DESTINATION/Pie
In  D:/Do

In  D:/Downloads/Analysed/DESTINATION/Chicken Biryani
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Biryani
In  D:/Downloads/Analysed/DESTINATION/Chicken Breast Roasted
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Breast Roasted
In  D:/Downloads/Analysed/DESTINATION/Chicken Curry
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Curry
In  D:/Downloads/Analysed/DESTINATION/Chicken Karahi
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Karahi
In  D:/Downloads/Analysed/DESTINATION/Chicken Keema Balls
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Keema Balls
In  D:/Downloads/Analysed/DESTINATION/Chicken Keema Paratha
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Keema Paratha
In  D:/Downloads/Analysed/DESTINATION/Chicken Lollipop
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Chicken Lollipop
In  D:/Downloads/Analysed/DESTINATION/Chicken Malai Tikka
Fo

In  D:/Downloads/Analysed/DESTINATION/Ice Gola
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Ice Gola
In  D:/Downloads/Analysed/DESTINATION/Iced Tea
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Iced Tea
In  D:/Downloads/Analysed/DESTINATION/Idiyappam
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Idiyappam
In  D:/Downloads/Analysed/DESTINATION/Idli
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Idli
In  D:/Downloads/Analysed/DESTINATION/Ilish Macher Jhol
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Ilish Macher Jhol
In  D:/Downloads/Analysed/DESTINATION/Jackfruit Curry
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Jackfruit Curry
In  D:/Downloads/Analysed/DESTINATION/Jadoh
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Jadoh
In  D:/Downloads/Analysed/DESTINATION/Jaggery Roti
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Jaggery Roti
In  D:/Downloads/Analysed/DESTINATION/Jalebi

In  D:/Downloads/Analysed/DESTINATION/Paneer Chili
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Paneer Chili
In  D:/Downloads/Analysed/DESTINATION/Paneer Pakoda
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Paneer Pakoda
In  D:/Downloads/Analysed/DESTINATION/Paneer Seekh Kebab
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Paneer Seekh Kebab
In  D:/Downloads/Analysed/DESTINATION/Pani Puri
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Pani Puri
In  D:/Downloads/Analysed/DESTINATION/Panipuri
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Panipuri
In  D:/Downloads/Analysed/DESTINATION/Panjiri
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Panjiri
In  D:/Downloads/Analysed/DESTINATION/Panneer Rool
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Panneer Rool
In  D:/Downloads/Analysed/DESTINATION/Papaya
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Papaya
In  D:/Downloads/Analysed/DEST

In  D:/Downloads/Analysed/DESTINATION/Veg Keema
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Veg Keema
In  D:/Downloads/Analysed/DESTINATION/Veg Koftacurry
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Veg Koftacurry
In  D:/Downloads/Analysed/DESTINATION/Veg Korma
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Veg Korma
In  D:/Downloads/Analysed/DESTINATION/Veg Manchurian Dry
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Veg Manchurian Dry
In  D:/Downloads/Analysed/DESTINATION/Veg Sandwitch
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Veg Sandwitch
In  D:/Downloads/Analysed/DESTINATION/Veg Steamed Momos
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Veg Steamed Momos
In  D:/Downloads/Analysed/DESTINATION/Vegetable Patties
Folder getting removed :  D:/Downloads/Analysed/DESTINATION/Vegetable Patties
In  D:/Downloads/Analysed/DESTINATION/Vermicelli Upma
Folder getting removed :  D:/Downloads/Analysed/DEST